In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

import torch
from torchvision import transforms
from sklearn.utils import shuffle
from skimage.metrics import structural_similarity as ssim
import pennylane as qml
# from livelossplot import PlotLosses
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
from sklearn.datasets import load_digits

%matplotlib inline

In [2]:
data = load_digits().data
classes = load_digits().target
filter = np.where((classes == 0 )|(classes == 1))
data, classes = data[filter], classes[filter]
data, classes = shuffle(data, classes)
data, classes = data[:45], classes[:45]
# print(data.shape, classes.shape)
x_aux = data
y = classes
x_main = list(range(len(data)))
for i in range(len(x_main)):
    x_main[i] = np.binary_repr(x_main[i], width=9)
    x_main[i] = [int(char) for char in x_main[i]]

# x_main = (x_main-np.mean(x_main))/np.std(x_main)
# x_aux = (x_aux -np.mean(x_aux))/np.std(x_aux)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 20

# DEFINE AUX PQC STRUCTURE AND DECLARE TORCHLAYER FOR IT
wires_aux_pqc = 6
layers_aux = 3
dev = qml.device("default.qubit", wires=9)

## Fixed AUX PQC + Trainable QRAM PQC

In [ ]:
# wires_aux_pqc = 6
# layers_aux = 3
# dev = qml.device("default.qubit", wires=9)

# rand_params = np.random.uniform(high=2 * np.pi, size=(layers_aux, 20))

# @qml.qnode(dev, interface='torch')
# def aux_pqc(inputs):
#     qml.AmplitudeEmbedding(features=inputs,normalize=True,wires=range(wires_aux_pqc))
#     seed = 42
#     qml.RandomLayers(weights=rand_params, wires=range(wires_aux_pqc),seed=seed)
#     return qml.probs(wires=range(wires_aux_pqc))

# wires_qcnn = 9
# layers_qcnn = 3
# @qml.qnode(dev, interface='torch')
# def qcnn(inputs, params, weights):
#     qml.AngleEmbedding(features=inputs, wires=range(wires_qcnn), rotation='Y')
#     # qml.BasisEmbedding(features=inputs, wires=range(wires_qcnn))
#     for l in range(layers_qcnn):
#         cnt = 0
#         # shape = qml.StronglyEntanglingLayers.shape(n_layers=layers_qcnn, n_wires=wires_qcnn)
#         # weights = np.random.random(size=shape)
#         for i in range(wires_qcnn - 1):
#             qml.RY(params[l*36+cnt],wires=i)
#             qml.RY(params[l*36+cnt+1],wires=i+1)
#             qml.CNOT(wires=[i,i+1])
#             qml.CRZ(params[l*36+cnt+2], wires=[i,i+1])
#             qml.PauliX(wires=i+1)
#             qml.CRX(params[l*36+cnt+3],wires=[i,i+1])
#             cnt = cnt + 4
#         for i in [wires_qcnn - 1]:
#             qml.RY(params[l*36+cnt],wires=i)
#             qml.RY(params[l*36+cnt+1],wires=i-(wires_qcnn - 1))
#             qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
#             qml.CRZ(params[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
#             qml.PauliX(wires=i-(wires_qcnn - 1))
#             qml.CRX(params[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
#             cnt = cnt + 4
#     qml.StronglyEntanglingLayers(weights=weights, wires=range(wires_qcnn))
#     return qml.probs(wires=[0,1,2,6,7,8])

# weights_qcnn = {"params":layers_qcnn*36, "weights":(layers_qcnn,wires_qcnn,layers_qcnn)}

# qcnn_main = qml.qnn.TorchLayer(qcnn, weights_qcnn, init_method=torch.nn.init.normal_)

# loss_function = torch.nn.MSELoss()
# # optimizer = torch.optim.SGD(qcnn_main.parameters(),lr=0.001,momentum=0.9,nesterov=True)
# optimizer = torch.optim.Adam(qcnn_main.parameters(),lr=0.01)

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# BATCH_SIZE = 20

# x_aux = torch.tensor(x_aux).to(DEVICE)
# y = torch.tensor(y,dtype=torch.double).to(DEVICE)
# x_main = torch.tensor(x_main, dtype=torch.float64).to(DEVICE)

# epochs = 20
# for epoch in range(epochs):
#     print("******EPOCH #" + str(epoch) + " START ******")
#     epoch_loss = torch.tensor([0], dtype=torch.float64).to(DEVICE)
#     running_acc = 0
#     for i in range(x_aux.shape[0]//BATCH_SIZE):
#         bs_counter = 0
#         optimizer.zero_grad()
#         loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
#         while bs_counter < BATCH_SIZE: 
#             # print(x_main[i*BATCH_SIZE+bs_counter])
#             ground_truth = aux_pqc(x_aux[i*BATCH_SIZE+bs_counter])
#             pred = qcnn_main(x_main[i*BATCH_SIZE+bs_counter])
#             # print(pred.tolist())
#             temp_loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
#             a = []
#             for j in range(len(pred)):
#                 temp_loss = temp_loss + loss_function(pred[j], ground_truth[j])
#                 a.append(torch.abs(pred[j]-ground_truth[j]).item())
#             loss = loss + temp_loss
#             epoch_loss = epoch_loss + loss
#             # loss = loss + loss_function(pred, y_train[i*BATCH_SIZE+bs_counter])
#             # print(temp_loss.item())
#             if temp_loss.item() < 0.05:
#                 running_acc = running_acc + 1
#             bs_counter = bs_counter + 1
#         loss = loss/BATCH_SIZE
#         # loss.requires_grad = True
#         loss.backward()
#         # for param in qcnn_main.parameters():
#         #     print(param.grad)
#         optimizer.step()
#         if i % 5 == 0 and i != 0:
#             # print(pred, y_train[i*BATCH_SIZE+bs_counter])
#             # for j in range(pred.shape[0]):
#             #     if j == 0:
#             #         # print(pred[j].item(), ground_truth[j].item())
#             #         print(pred)
#             print("Running Accuracy: " + str(running_acc/((i+1)*BATCH_SIZE)*100) + "%")
#     print("EPOCH LOSS:" + str(epoch_loss.item()))
#     print("******EPOCH #" + str(epoch) + " END ******")

## Trainable Aux PQC + Trainable QRAM PQC

In [3]:
@qml.qnode(dev, interface='torch')
def aux_pqc(inputs, params):
    qml.AmplitudeEmbedding(features=inputs,normalize=True,wires=range(wires_aux_pqc))
    seed = 42
    qml.RandomLayers(weights=params, wires=range(wires_aux_pqc),seed=seed)
    return qml.probs(wires=range(wires_aux_pqc))

weights_aux = {'params':(layers_aux, 20)}

qcnn_aux = qml.qnn.TorchLayer(aux_pqc, weights_aux, init_method=torch.nn.init.normal_)

# DEFINE MAIN QRAM PQC STRUCTURE AND DECLARE TORCHLAYER FOR IT
wires_qcnn = 9
layers_qcnn = 3
@qml.qnode(dev, interface='torch')
def qcnn(inputs, params, weights):
    qml.AngleEmbedding(features=inputs, wires=range(wires_qcnn), rotation='Y')
    # qml.BasisEmbedding(features=inputs, wires=range(wires_qcnn))
    for l in range(layers_qcnn):
        cnt = 0
        #shape = qml.StronglyEntanglingLayers.shape(n_layers=layers_qcnn, n_wires=wires_qcnn)
        #weights = np.random.random(size=shape)
        for i in range(wires_qcnn - 1):
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    qml.StronglyEntanglingLayers(weights=weights, wires=range(wires_qcnn))
    return qml.probs(wires=[0,1,2,6,7,8])

weights_qcnn = {"params":layers_qcnn*36, "weights":(layers_qcnn,wires_qcnn,layers_qcnn)}

qcnn_main = qml.qnn.TorchLayer(qcnn, weights_qcnn, init_method=torch.nn.init.normal_)

# DEFINE OVERALL QRAM CLASS CONSISTING OF BOTH AUX PQC AND MAIN PQC
class QRAM(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.qcnn_aux = torch.nn.Sequential(qcnn_aux)
        self.qcnn_main = torch.nn.Sequential(qcnn_main)
    
    def forward(self,x_aux,x_main):
        ground_truth = self.qcnn_aux(x_aux)
        pred = self.qcnn_main(x_main)
        return ground_truth, pred

qram = QRAM().to(DEVICE).float()

loss_function = torch.nn.MSELoss()
# optimizer = torch.optim.SGD(qcnn_main.parameters(),lr=0.001,momentum=0.9,nesterov=True)
optimizer = torch.optim.Adam(qram.parameters(),lr=0.01)

x_aux = torch.tensor(x_aux).to(DEVICE)
y = torch.tensor(y,dtype=torch.double).to(DEVICE)
x_main = torch.tensor(x_main, dtype=torch.float64).to(DEVICE)

# TRAIN AUX PQC AND MAIN PQC TOGETHER
epochs = 5
for epoch in range(epochs):
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    running_acc = 0
    for i in range(x_aux.shape[0]//BATCH_SIZE):
        bs_counter = 0
        optimizer.zero_grad()
        loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            # print(x_main[i*BATCH_SIZE+bs_counter])
            # ground_truth = aux_pqc(x_aux[i*BATCH_SIZE+bs_counter])
            # pred = qcnn_main(x_main[i*BATCH_SIZE+bs_counter])
            ground_truth, pred  = qram(x_aux[i*BATCH_SIZE+bs_counter],x_main[i*BATCH_SIZE+bs_counter])
            # print(pred.tolist())
            temp_loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            a = []
            for j in range(len(pred)):
                temp_loss = temp_loss + loss_function(pred[j], ground_truth[j])
                a.append(torch.abs(pred[j]-ground_truth[j]).item())
            loss = loss + temp_loss
            epoch_loss = epoch_loss + loss
            # loss = loss + loss_function(pred, y_train[i*BATCH_SIZE+bs_counter])
            # print(temp_loss.item())
            if temp_loss.item() < 0.05:
                running_acc = running_acc + 1
            bs_counter = bs_counter + 1
        loss = loss/BATCH_SIZE
        # loss.requires_grad = True
        loss.backward()
        # for param in qcnn_main.parameters():
        #     print(param.grad)
        optimizer.step()
        if i % 5 == 0 and i != 0:
            # print(pred, y_train[i*BATCH_SIZE+bs_counter])
            # for j in range(pred.shape[0]):
            #     print(pred[j].item(), ground_truth[j].item())
            print("Running Accuracy: " + str(running_acc/((i+1)*BATCH_SIZE)*100) + "%")
    print("EPOCH LOSS:" + str(epoch_loss.item()))
    print("******EPOCH #" + str(epoch) + " END ******")

print("****")
print()
print()
print()
print("****")

# TRAIN ONLY MAIN PQC AND KEEP TRAINED AUX PQC PARAMETERS FIXED
epochs = 5
for epoch in range(epochs):
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    running_acc = 0
    for i in range(x_aux.shape[0]//BATCH_SIZE):
        bs_counter = 0
        optimizer.zero_grad()
        loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            # print(x_main[i*BATCH_SIZE+bs_counter])
            # ground_truth = aux_pqc(x_aux[i*BATCH_SIZE+bs_counter])
            # pred = qcnn_main(x_main[i*BATCH_SIZE+bs_counter])
            ground_truth, pred  = qram(x_aux[i*BATCH_SIZE+bs_counter],x_main[i*BATCH_SIZE+bs_counter])
            # print(pred.tolist())
            temp_loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            a = []
            for j in range(len(pred)):
                temp_loss = temp_loss + loss_function(pred[j], ground_truth[j])
                a.append(torch.abs(pred[j]-ground_truth[j]).item())
            loss = loss + temp_loss
            epoch_loss = epoch_loss + loss
            # loss = loss + loss_function(pred, y_train[i*BATCH_SIZE+bs_counter])
            # print(temp_loss.item())
            if temp_loss.item() < 0.05:
                running_acc = running_acc + 1
            bs_counter = bs_counter + 1
        loss = loss/BATCH_SIZE
        # loss.requires_grad = True
        loss.backward()
        for param in qcnn_main.parameters():
            if param.shape == (layers_aux, 20):
                param.grad = torch.zeros((layers_aux, 20), dtype=torch.float64)
        optimizer.step()
        if i % 5 == 0 and i != 0:
            # print(pred, y_train[i*BATCH_SIZE+bs_counter])
            # for j in range(pred.shape[0]):
            #     print(pred[j].item(), ground_truth[j].item())
            print("Running Accuracy: " + str(running_acc/((i+1)*BATCH_SIZE)*100) + "%")
    print("EPOCH LOSS:" + str(epoch_loss.item()))
    print("******EPOCH #" + str(epoch) + " END ******")




******EPOCH #0 START ******
EPOCH LOSS:6.656999490819594
******EPOCH #0 END ******
******EPOCH #1 START ******
EPOCH LOSS:5.600795601888476
******EPOCH #1 END ******
******EPOCH #2 START ******
EPOCH LOSS:4.880166569267084
******EPOCH #2 END ******
******EPOCH #3 START ******
EPOCH LOSS:4.3881093049327
******EPOCH #3 END ******
******EPOCH #4 START ******
EPOCH LOSS:4.04250681705732
******EPOCH #4 END ******
****



****
******EPOCH #0 START ******
EPOCH LOSS:3.7795817735565844
******EPOCH #0 END ******
******EPOCH #1 START ******
EPOCH LOSS:3.5746346251073726
******EPOCH #1 END ******
******EPOCH #2 START ******
EPOCH LOSS:3.423576469004699
******EPOCH #2 END ******
******EPOCH #3 START ******
EPOCH LOSS:3.313613430591445
******EPOCH #3 END ******
******EPOCH #4 START ******
EPOCH LOSS:3.224138983846711
******EPOCH #4 END ******


In [33]:
# AUGMENT QNN CLASSIFIER TO THE QRAM AND TRAIN THE QNN CLASSIFIER BY KEEPING TRAINED QRAM PARAMETERS FIXED
layers_classifier = 10
@qml.qnode(dev, interface='torch')
def quantum_ram_classifier(inputs, params, weights, params2):
    qml.AngleEmbedding(features=inputs, wires=range(wires_qcnn), rotation='Y')
    # QRAM STRUCTURE
    for l in range(layers_qcnn):
        cnt = 0
        for i in range(wires_qcnn - 1):
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    qml.StronglyEntanglingLayers(weights=weights, wires=range(wires_qcnn))
    # QNN CLASSIFIER STRUCTURE
    for l in range(layers_classifier):
        cnt = 0
        for i in range(wires_qcnn - 1):
            qml.RY(params2[l*36+cnt],wires=i)
            qml.RY(params2[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params2[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params2[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params2[l*36+cnt],wires=i)
            qml.RY(params2[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params2[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params2[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    return qml.expval(qml.PauliZ(0)) 

weights_qram_classifier = {"params":layers_qcnn*36, "weights":(layers_qcnn,wires_qcnn,layers_qcnn), "params2":layers_classifier*36}

qram_classifier = qml.qnn.TorchLayer(quantum_ram_classifier, weights_qram_classifier, init_method=torch.nn.init.uniform_)

optimizer_classifier = torch.optim.Adam(qram_classifier.parameters(),lr=0.1)

trained_qram_params = list(qcnn_main.parameters())
# print(trained_qram_params[0])

with torch.no_grad():
    cnt = 0
    for param in qram_classifier.parameters():
        if cnt < 2:
            param = trained_qram_params[cnt].data
            param.requires_grad = False
        # print(param)
        cnt = cnt + 1

epochs = 20
for epoch in range(epochs):
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    running_acc = 0
    for i in range(x_aux.shape[0]//BATCH_SIZE):
        bs_counter = 0
        optimizer_classifier.zero_grad()
        loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            out = (qram_classifier(x_main[i*BATCH_SIZE+bs_counter])+1.0)/2
            # print(qram_classifier(x_main[i*BATCH_SIZE+bs_counter]).item(), qram_classifier(x_main[i*BATCH_SIZE+bs_counter]).item()+1.0,(qram_classifier(x_main[i*BATCH_SIZE+bs_counter]).item()+1.0)/2, y[i*BATCH_SIZE+bs_counter].item())
            # print(out < torch.tensor(0, dtype=torch.float64))
            # if out < torch.tensor(0, dtype=torch.float64):
            #     out = torch.tensor(0,dtype=torch.float64, requires_grad=True).to(DEVICE)
            # else:
            #     out = torch.tensor(1, dtype=torch.float64, requires_grad=True).to(DEVICE)
            # temp_loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            # temp_loss = temp_loss + loss_function(out, y[i*BATCH_SIZE+bs_counter])
            loss = loss + loss_function((qram_classifier(x_main[i*BATCH_SIZE+bs_counter])+1.0)/2, y[i*BATCH_SIZE+bs_counter])
            # print(out.item(), y[i*BATCH_SIZE+bs_counter].item())
            epoch_loss = epoch_loss + loss
            if loss_function((qram_classifier(x_main[i*BATCH_SIZE+bs_counter])+1.0)/2, y[i*BATCH_SIZE+bs_counter]).item() < 0.25:
                running_acc = running_acc + 1
            bs_counter = bs_counter + 1
            print(qram_classifier(x_main[i*BATCH_SIZE+bs_counter]).item(), (qram_classifier(x_main[i*BATCH_SIZE+bs_counter]).item()+1.0)/2, y[i*BATCH_SIZE+bs_counter].item())
        loss = loss/BATCH_SIZE
        loss.backward()
        cnt = 0
        for param in qram_classifier.parameters():
            # if cnt < 2:
            #     param.grad = torch.zeros(param.shape, dtype=param.grad.dtype)
            # cnt = cnt + 1
            # print(param.grad)
            pass
        optimizer_classifier.step()
        if i % 1 == 0 and i != 0:
            print("Running Accuracy: " + str(running_acc/((i+1)*BATCH_SIZE)*100) + "%")
    print("EPOCH LOSS:" + str(epoch_loss.item()))
    print("******EPOCH #" + str(epoch) + " END ******")
            
            

******EPOCH #0 START ******
-0.008706259537270333 0.49564687023136483 1.0
-0.0017319088647679837 0.499134045567616 1.0
0.0003663020606930467 0.5001831510303465 0.0
0.03452505061899336 0.5172625253094967 0.0
0.015803400280263225 0.5079017001401316 1.0
0.06571744635800303 0.5328587231790015 1.0
0.03620575896898515 0.5181028794844926 1.0
-0.07197567770962554 0.46401216114518723 1.0
-0.04063101298229416 0.4796844935088529 0.0
-0.028380417380190037 0.485809791309905 0.0
-0.02849727857588802 0.485751360712056 1.0
-0.008192719284353833 0.4959036403578231 0.0
-0.013550169057151784 0.4932249154714241 0.0
0.029528104533305677 0.5147640522666528 0.0
0.007430806364427833 0.503715403182214 1.0
-0.023412539585336845 0.4882937302073316 1.0
-0.03882659053255366 0.48058670473372317 0.0
-0.019185290657989573 0.4904073546710052 1.0
-0.03624074458824211 0.48187962770587894 0.0
-0.005030204285070894 0.4974848978574645 1.0
0.012395346018350906 0.5061976730091755 1.0
-0.029076462247055346 0.4854617688764723 

# Classification Without QRAM 

In [32]:
layers_classifier = 5
@qml.qnode(dev, interface='torch')
def quantum_classifier(inputs, params):
    qml.AmplitudeEmbedding(features=inputs, wires=range(wires_qcnn-3), normalize=True)
    for l in range(layers_classifier):
        cnt = 0
        for i in range(wires_qcnn - 1):
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    return qml.expval(qml.PauliZ(0))

weights_quantum_classifier = {"params":layers_classifier*36}
q_classifier = qml.qnn.TorchLayer(quantum_classifier, weights_quantum_classifier, init_method=torch.nn.init.normal_)

optimizer_classifier = torch.optim.Adam(q_classifier.parameters(),lr=0.01)

epochs = 20
for epoch in range(epochs):
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    running_acc = 0
    for i in range(x_aux.shape[0]//BATCH_SIZE):
        bs_counter = 0
        optimizer_classifier.zero_grad()
        loss2 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            out = (q_classifier(x_aux[i*BATCH_SIZE+bs_counter])+1)/2
            # print(out.item(), q_classifier(x_aux[i*BATCH_SIZE+bs_counter]).item()+1)
            # temp_loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            # temp_loss = temp_loss + loss_function(out, y[i*BATCH_SIZE+bs_counter])
            loss2 = loss2 + loss_function((q_classifier(x_aux[i*BATCH_SIZE+bs_counter])+1)/2, y[i*BATCH_SIZE+bs_counter])
            # print(out.item(), y[i*BATCH_SIZE+bs_counter].item())
            epoch_loss = epoch_loss + loss2
            if loss_function((q_classifier(x_aux[i*BATCH_SIZE+bs_counter])+1)/2, y[i*BATCH_SIZE+bs_counter]).item() < 0.25:
                running_acc = running_acc + 1
            bs_counter = bs_counter + 1
            print(q_classifier(x_aux[i*BATCH_SIZE+bs_counter]).item(), (q_classifier(x_aux[i*BATCH_SIZE+bs_counter]).item()+1)/2, y[i*BATCH_SIZE+bs_counter].item())
        loss2 = loss2/BATCH_SIZE
        loss2.backward()
        optimizer_classifier.step()
        if i % 1 == 0 and i != 0:
            print("Running Accuracy: " + str(running_acc/((i+1)*BATCH_SIZE)*100) + "%")
    print("EPOCH LOSS:" + str(epoch_loss.item()))
    print("******EPOCH #" + str(epoch) + " END ******")
    

******EPOCH #0 START ******
0.09492901525512754 0.5474645076275637 1.0
0.027854178241309813 0.5139270891206549 1.0
0.009757902163612386 0.5048789510818061 0.0
0.0028918774307795103 0.5014459387153898 0.0
0.10274883478540742 0.5513744173927038 1.0
0.05796552047064518 0.5289827602353225 1.0
0.08626791100543141 0.5431339555027157 1.0
0.07702714086063922 0.5385135704303197 1.0
0.051613861397529714 0.5258069306987648 0.0
0.0469964987726581 0.523498249386329 0.0
0.1113704432026954 0.5556852216013477 1.0
0.05407236791994008 0.52703618395997 0.0
-0.001306173994141624 0.4993469130029292 0.0
0.000916351747035582 0.5004581758735178 0.0
0.07284224611853951 0.5364211230592697 1.0
0.06613776701352347 0.5330688835067617 1.0
0.00024714416878113266 0.5001235720843906 0.0
0.07603103745517137 0.5380155187275857 1.0
0.018153264206618835 0.5090766321033094 0.0
0.015394370428004123 0.5076971852140021 1.0
0.1169095037963318 0.558454751898166 1.0
0.000543807843747024 0.5002719039218735 0.0
-0.0023881383207303

In [ ]:
# for param in qram.parameters():
#     print(param.shape)
# print(list(qram.parameters()))

In [ ]:
# for j in range(pred.shape[0]):
#     print(pred[j].item(), ground_truth[j].item())
# x_test = torch.tensor(x_test).to(DEVICE)
# y_test = torch.tensor(y_test,dtype=torch.double).to(DEVICE)

# acc = 0
# for i in range(x_test.shape[0]):
#     temp_loss = 0
#     for j in range(4):
#         temp_loss = temp_loss + loss_function(pred[j], y_test[i][j])
#     if temp_loss.item() < 1:
#         acc += 1

# print("Testing Accuracy: " + str(acc/x_test.shape[0]*100)) 